In [33]:
import pandas as pd
import numpy as np
from env import get_connection
from env import username, hostname, password
import os
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")

In [2]:
def get_zillow():
    filename = "zillow.csv"

    if os.path.isfile(filename):
        return pd.read_csv(filename)
    else:
        # read the SQL query into a dataframe
        df = pd.read_sql('''SELECT
                prop.*,
                predictions_2017.logerror,
                predictions_2017.transactiondate,
                air.airconditioningdesc,
                arch.architecturalstyledesc,
                build.buildingclassdesc,
                heat.heatingorsystemdesc,
                landuse.propertylandusedesc,
                story.storydesc,
                construct.typeconstructiondesc
                FROM properties_2017 prop
                JOIN (SELECT parcelid, MAX(transactiondate) AS max_transactiondate
                FROM predictions_2017
                GROUP BY parcelid) pred USING(parcelid)
                JOIN predictions_2017 ON pred.parcelid = predictions_2017.parcelid AND pred.max_transactiondate = predictions_2017.transactiondate
                LEFT JOIN airconditioningtype air USING (airconditioningtypeid)
                LEFT JOIN architecturalstyletype arch USING (architecturalstyletypeid)
                LEFT JOIN buildingclasstype build USING (buildingclasstypeid)
                LEFT JOIN heatingorsystemtype heat USING (heatingorsystemtypeid)
                LEFT JOIN propertylandusetype landuse USING (propertylandusetypeid)
                LEFT JOIN storytype story USING (storytypeid)
                LEFT JOIN typeconstructiontype construct USING (typeconstructiontypeid)
                WHERE prop.latitude IS NOT NULL
                AND prop.longitude IS NOT NULL
                AND transactiondate <= '2017-12-31'
                AND propertylandusedesc = "Single Family Residential" ''')
        ,get_connection('zillow')

        # Write that dataframe to disk for later. Called "caching" the data for later.
        df.to_csv(filename)

        # Return the dataframe to the calling code
        return df 

In [3]:
df= get_zillow()
df

,parcelid,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,id,basementsqft,...,id.1,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,14297519,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1727539,NaN,...,0,0.025595,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
1,17052889,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1387261,NaN,...,1,0.055619,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
2,14186244,NaN,NaN,261.0,NaN,NaN,NaN,NaN,11677,NaN,...,2,0.005383,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
3,12177905,NaN,NaN,261.0,2.0,NaN,NaN,NaN,2288172,NaN,...,3,-0.103410,2017-01-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
4,10887214,NaN,NaN,266.0,2.0,NaN,NaN,1.0,1970746,NaN,...,4,0.006940,2017-01-01,Central,NaN,NaN,Central,Condominium,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77574,10833991,NaN,NaN,266.0,2.0,NaN,NaN,1.0,2864704,NaN,...,77608,-0.002245,2017-09-20,Central,NaN,NaN,Central,Condominium,NaN,NaN
77575,11000655,NaN,NaN,261.0,2.0,NaN,NaN,NaN,673515,NaN,...,77609,0.020615,2017-09-20,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
77576,17239384,NaN,NaN,261.0,NaN,NaN,NaN,NaN,2968375,NaN,...,77610,0.013209,2017-09-21,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
77577,12773139,NaN,NaN,261.0,2.0,NaN,NaN,1.0,1843709,NaN,...,77611,0.037129,2017-09-21,Central,NaN,NaN,Central,Single Family Residential,NaN,NaN


In [4]:
df.describe()

,parcelid,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,id,basementsqft,...,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock,id.1,logerror
count,7.757900e+04,223.000000,50.0,77579.000000,49571.000000,15.000000,207.000000,25007.000000,7.757900e+04,50.000000,...,172.0,7.746400e+04,7.757800e+04,77579.0,7.757700e+04,77574.000000,2900.000000,7.733200e+04,77579.000000,77579.000000
mean,1.300828e+07,6.040359,7.0,261.824476,3.921749,3.933333,7.386473,1.812013,1.495392e+06,679.720000,...,1.0,1.892811e+05,4.901506e+05,2016.0,3.011515e+05,5995.961788,14.088276,6.049667e+13,38806.223579,0.016803
std,3.519399e+06,0.556035,0.0,5.141596,3.594780,0.258199,2.728030,2.965768,8.609686e+05,689.703546,...,0.0,2.304107e+05,6.537979e+05,0.0,4.927249e+05,7628.859728,2.181281,1.533339e+12,22403.467491,0.170738
min,1.071186e+07,4.000000,7.0,31.000000,1.000000,3.000000,2.000000,1.000000,3.490000e+02,38.000000,...,1.0,4.400000e+01,1.000000e+03,2016.0,1.610000e+02,19.920000,3.000000,6.037101e+13,0.000000,-4.655420
25%,1.153820e+07,6.000000,7.0,261.000000,2.000000,4.000000,7.000000,1.000000,7.521420e+05,273.000000,...,1.0,8.417925e+04,2.068985e+05,2016.0,8.529300e+04,2712.630000,14.000000,6.037311e+13,19404.500000,-0.024309
50%,1.253054e+07,6.000000,7.0,261.000000,2.000000,4.000000,7.000000,1.000000,1.498227e+06,515.000000,...,1.0,1.364045e+05,3.588785e+05,2016.0,2.031880e+05,4448.265000,15.000000,6.037603e+13,38804.000000,0.006676
75%,1.421136e+07,6.000000,7.0,266.000000,7.000000,4.000000,7.000000,1.000000,2.240879e+06,796.500000,...,1.0,2.187340e+05,5.690000e+05,2016.0,3.667530e+05,6926.917500,15.000000,6.059042e+13,58207.500000,0.039291
max,1.676893e+08,13.000000,7.0,275.000000,24.000000,4.000000,21.000000,13.000000,2.982274e+06,3560.000000,...,1.0,1.142179e+07,4.906124e+07,2016.0,4.895220e+07,586639.300000,99.000000,4.830301e+14,77612.000000,5.262999


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77579 entries, 0 to 77578
Data columns (total 69 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      77579 non-null  int64  
 1   typeconstructiontypeid        223 non-null    float64
 2   storytypeid                   50 non-null     float64
 3   propertylandusetypeid         77579 non-null  float64
 4   heatingorsystemtypeid         49571 non-null  float64
 5   buildingclasstypeid           15 non-null     float64
 6   architecturalstyletypeid      207 non-null    float64
 7   airconditioningtypeid         25007 non-null  float64
 8   id                            77579 non-null  int64  
 9   basementsqft                  50 non-null     float64
 10  bathroomcnt                   77579 non-null  float64
 11  bedroomcnt                    77579 non-null  float64
 12  buildingqualitytypeid         49809 non-null  float64
 13  c

In [6]:
df.value_counts()

Series([], dtype: int64)

In [7]:
df.dtypes

parcelid                    int64
typeconstructiontypeid    float64
storytypeid               float64
propertylandusetypeid     float64
heatingorsystemtypeid     float64
                           ...   
buildingclassdesc          object
heatingorsystemdesc        object
propertylandusedesc        object
storydesc                  object
typeconstructiondesc       object
Length: 69, dtype: object

In [8]:
df.shape

(77579, 69)

In [9]:
df[df.parcelid == ''].sum()

parcelid                  0.0
typeconstructiontypeid    0.0
storytypeid               0.0
propertylandusetypeid     0.0
heatingorsystemtypeid     0.0
                         ... 
buildingclassdesc         0.0
heatingorsystemdesc       0.0
propertylandusedesc       0.0
storydesc                 0.0
typeconstructiondesc      0.0
Length: 69, dtype: float64

## Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [10]:
null_col= df.isnull().sum()    
perc_null= null_col / df.shape[0] * 100 

In [11]:
def the_perc(df):   
    null_col= df.isnull().sum()    
    perc_null= null_col / df.shape[0] * 100 
    nulldf= pd.DataFrame(
    {
    'null_count': null_col,
    'null_percentage': perc_null})
    return nulldf

In [12]:
df.parcelid.isnull().sum()

0

In [13]:
null_col

parcelid                      0
typeconstructiontypeid    77356
storytypeid               77529
propertylandusetypeid         0
heatingorsystemtypeid     28008
                          ...  
buildingclassdesc         77564
heatingorsystemdesc       28008
propertylandusedesc           0
storydesc                 77529
typeconstructiondesc      77356
Length: 69, dtype: int64

In [14]:
the_perc(df)

,null_count,null_percentage
parcelid,0,0.000000
typeconstructiontypeid,77356,99.712551
storytypeid,77529,99.935550
propertylandusetypeid,0,0.000000
heatingorsystemtypeid,28008,36.102554
...,...,...
buildingclassdesc,77564,99.980665
heatingorsystemdesc,28008,36.102554
propertylandusedesc,0,0.000000
storydesc,77529,99.935550


# Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer.



In [15]:
df= df[df.propertylandusedesc == 'Single Family Residential']
df

,parcelid,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,id,basementsqft,...,id.1,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,14297519,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1727539,NaN,...,0,0.025595,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
1,17052889,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1387261,NaN,...,1,0.055619,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
2,14186244,NaN,NaN,261.0,NaN,NaN,NaN,NaN,11677,NaN,...,2,0.005383,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
3,12177905,NaN,NaN,261.0,2.0,NaN,NaN,NaN,2288172,NaN,...,3,-0.103410,2017-01-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
6,12095076,NaN,NaN,261.0,2.0,NaN,NaN,1.0,781532,NaN,...,6,-0.001011,2017-01-01,Central,NaN,NaN,Central,Single Family Residential,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77573,12412492,NaN,NaN,261.0,2.0,NaN,NaN,NaN,2274245,NaN,...,77607,0.001082,2017-09-19,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
77575,11000655,NaN,NaN,261.0,2.0,NaN,NaN,NaN,673515,NaN,...,77609,0.020615,2017-09-20,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
77576,17239384,NaN,NaN,261.0,NaN,NaN,NaN,NaN,2968375,NaN,...,77610,0.013209,2017-09-21,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
77577,12773139,NaN,NaN,261.0,2.0,NaN,NaN,1.0,1843709,NaN,...,77611,0.037129,2017-09-21,Central,NaN,NaN,Central,Single Family Residential,NaN,NaN


# Mall Customers

1. Acquire data from the customers table in the mall_customers database.

In [16]:
def get_mall():
    '''
    Pulls data from codeup database and writes into a dataframe
    '''
    url = f"mysql+pymysql://{username}:{password}@{hostname}/mall_customers"
    
    query = '''
            SELECT * FROM customers
            '''

    df = pd.read_sql(query, url)

    return df


In [17]:
df= get_mall()
df

,customer_id,gender,age,annual_income,spending_score
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40
...,...,...,...,...,...
195,196,Female,35,120,79
196,197,Female,45,126,28
197,198,Male,32,126,74
198,199,Male,32,137,18


2. Summarize the data (include distributions and descriptive statistics).

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   customer_id     200 non-null    int64 
 1   gender          200 non-null    object
 2   age             200 non-null    int64 
 3   annual_income   200 non-null    int64 
 4   spending_score  200 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 7.9+ KB


In [19]:
df.describe()

,customer_id,age,annual_income,spending_score
count,200.000000,200.000000,200.000000,200.000000
mean,100.500000,38.850000,60.560000,50.200000
std,57.879185,13.969007,26.264721,25.823522
min,1.000000,18.000000,15.000000,1.000000
25%,50.750000,28.750000,41.500000,34.750000
50%,100.500000,36.000000,61.500000,50.000000
75%,150.250000,49.000000,78.000000,73.000000
max,200.000000,70.000000,137.000000,99.000000


In [20]:
df.isna().sum()

customer_id       0
gender            0
age               0
annual_income     0
spending_score    0
dtype: int64

In [21]:
df.value_counts()

customer_id  gender  age  annual_income  spending_score
1            Male    19   15             39                1
138          Male    32   73             73                1
128          Male    40   71             95                1
129          Male    59   71             11                1
130          Male    38   71             75                1
                                                          ..
70           Female  32   48             47                1
71           Male    70   49             55                1
72           Female  47   49             42                1
73           Female  60   50             49                1
200          Male    30   137            83                1
Length: 200, dtype: int64

In [22]:
df.dtypes

customer_id        int64
gender            object
age                int64
annual_income      int64
spending_score     int64
dtype: object

3. Detect outliers using IQR.

In [23]:
col= ['annual_income', 'spending_score']

In [24]:
q1, q3= df.spending_score.quantile((0.25, 0.75))
q1, q3

(34.75, 73.0)

In [42]:
#See the quantiles for age
df['annual_income'].quantile([0.25, 0.75])

#Store the quantiles in variables
ai_q1, ai_q3 = df['annual_income'].quantile([0.25, 0.75])

#Calculate the IQR
ai_iqr = ai_q3 - ai_q1

#Calculate upper and lower bounds based on a k value of 1.5
ai_upper = ai_q3 + (ai_iqr * 1.5)
ai_lower = ai_q1 - (ai_iqr * 1.5)

#Check for outliers in the age column
df[df['annual_income'] > ai_upper]

,customer_id,gender,age,annual_income,spending_score
198,199,Male,32,137,18
199,200,Male,30,137,83


4. Split data into train, validate, and test.

In [47]:
def split_data(df, target=''):
        train, test = train_test_split(df, 
                               train_size = 0.8,
                               random_state=1349,
                              stratify=df[target])
        train, val = train_test_split(train,
                             train_size = 0.7,
                             random_state=1349,
                             stratify=train[target])
        print('Train shape:' ), print(train.shape)
    
        print('Validate shape:' ), print(val.shape)
    
        print('Test shape:' ), print(test.shape)
        return train, val, test

In [49]:
def the_split(df):

    train_validate, test= train_test_split(df, test_size= .2, random_state= 123)
    train, val= train_test_split(train_validate, test_size= .3, random_state =123)

    print('Train shape:' ), print(train.shape)
    
    print('Validate shape:' ), print(val.shape)
    
    print('Test shape:' ), print(test.shape)
    
    return train, val, test    


In [51]:
train, val, test= the_split(df)

Train shape:
(112, 5)
Validate shape:
(48, 5)
Test shape:
(40, 5)


5. Encode categorical columns using a one hot encoder (pd.get_dummies).

In [67]:
train = pd.get_dummies(train)
train.drop(columns=['gender_Male'], inplace=True)
train

,customer_id,age,annual_income,spending_score,gender_Female
123,124,39,69,91,0
76,77,45,54,53,1
171,172,28,87,75,0
10,11,67,19,14,0
35,36,21,33,81,1
...,...,...,...,...,...
175,176,30,88,86,1
101,102,49,62,48,1
148,149,34,78,22,1
99,100,20,61,49,0


6. Handles missing values.

In [73]:
def handle_missing_values(df, prop_required_column, prop_required_row):
    
    prop_null_column = 1 - prop_required_column
    
    for col in list(df.columns):
        
        null_sum = df[col].isna().sum()
        null_pct = null_sum / df.shape[0]
        
        if null_pct > prop_null_column:
            df.drop(columns=col, inplace=True)
            
    threshold = int(prop_required_row * df.shape[1])
    
    df.dropna(axis=0, thresh=threshold, inplace=True)
    
    return df

In [74]:
handle_missing_values(df)

TypeError: handle_missing_values() missing 2 required positional arguments: 'prop_required_column' and 'prop_required_row'

7. Scaling